In [1]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

Logging hadn't been started.


C:\Users\Henry\anaconda3\lib\site-packages\IPython\core\magics\logging.py:130: UserWarning: Couldn't start log: [Errno 2] No such file or directory: 'C:\\Users\\Henry/.logs/dw.py'
  warn("Couldn't start log: %s" % sys.exc_info()[1])


In [7]:
from static_grader import grader

ModuleNotFoundError: No module named 'static_grader'

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [8]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

A subdirectory or file dw-data already exists.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [9]:
import pandas as pd
import numpy as np
import gzip

In [10]:
# load the 2017 data
scripts_file = gzip.open('./dw-data/201701scripts_sample.csv.gz', 'r')
scripts = pd.read_csv(scripts_file)
scripts.head()

FileNotFoundError: [Errno 2] No such file or directory: './dw-data/201701scripts_sample.csv.gz'

In [11]:
scripts_file_16 = gzip.open('./dw-data/201606scripts_sample.csv.gz', 'r')
scripts_16 = pd.read_csv(scripts_file_16)
scripts_16.head()

FileNotFoundError: [Errno 2] No such file or directory: './dw-data/201606scripts_sample.csv.gz'

In [ ]:
practices_file =  gzip.open('./dw-data/practices.csv.gz', 'r')
col_names = [ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv(practices_file, names = col_names)
practices.head()

In [ ]:
chem_file =  gzip.open('./dw-data/chem.csv.gz', 'r')
chem = pd.read_csv(chem_file)
chem.head()

Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [ ]:
scripts.head()

In [ ]:
scripts_sum = scripts[['items', 'nic', 'act_cost', 'quantity']].agg(['sum'])
scripts_sum

In [ ]:
scripts_describe = scripts.describe()
scripts_describe

In [ ]:
scripts_output = pd.concat([scripts_sum, scripts_describe.drop(['count', 'min', 'max'])])
scripts_output

In [ ]:
items_list = []
for i in scripts_output['items']:
    items_list.append(i)
    
nic_list = []
for i in scripts_output['nic']:
    nic_list.append(i)
    
act_cost_list = []
for i in scripts_output['act_cost']:
    act_cost_list.append(i)
    
quantity_list = []
for i in scripts_output['quantity']:
    quantity_list.append(i)
    
output_list = [('items', tuple(items_list)), ('nic', tuple(nic_list)), ('act_cost', tuple(act_cost_list)), ('quantity', tuple(quantity_list))]
print(output_list)

In [ ]:
summary_stats = output_list

In [ ]:
grader.score.dw__summary_statistics(summary_stats)

## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [ ]:
scripts.head()

In [ ]:
scripts_gp = scripts.groupby('bnf_name')['items'].count()
scripts_gp

In [ ]:
sums = scripts.groupby('bnf_name').sum()['items']
sums.head()

In [ ]:
def get_most_common_item():
    sums = scripts.groupby('bnf_name').sum()['items']
    item = sums.idxmax(), sums.max()
    return [item]

In [ ]:
get_most_common_item()

In [ ]:
most_common_item = get_most_common_item()

In [ ]:
grader.score.dw__most_common_item(most_common_item)

## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [ ]:
practices.head()

In [ ]:
scripts.head()

In [ ]:
sorted_list = practices.sort_values('post_code').drop_duplicates('code', keep = 'first')
sorted_list.head()

In [ ]:
merged_list = scripts.merge(sorted_list, left_on = 'practice', right_on = 'code')
merged_list.head()

In [ ]:
total_items_by_postcode = merged_list.groupby(['post_code', 'bnf_name'])[['items']].sum()
total_items_by_postcode.head()

In [ ]:
total_items_by_postcode.reset_index(inplace = True)
total_items_by_postcode.head()

In [ ]:
total_items_max = total_items_by_postcode.groupby('post_code')[['items']].max()
total_items_max.head()

In [ ]:
total_items_max.reset_index(inplace = True)
total_items_max.head()

In [ ]:
total_items = total_items_max.merge(total_items_by_postcode, on = ['post_code', 'items'], how = 'left')
total_items.head()

In [ ]:
total_items_sorted = total_items.sort_values('bnf_name').drop_duplicates(['post_code', 'items'], keep = 'first')
total_items_sorted.head()

In [ ]:
total_items_sorted.sort_values('post_code', inplace = True)
total_items_sorted.head()

In [ ]:
total_items_sorted.shape

In [ ]:
_sum = total_items_by_postcode.groupby('post_code')[['items']].sum().reset_index()
_sum.head()

In [ ]:
_sum.shape

In [ ]:
final_df = _sum.merge(total_items_sorted, on = 'post_code')
final_df.head()

In [ ]:
final_df['amt%'] = final_df['items_y'].astype(float) / final_df['items_x'].astype(float)
final_df['amt%'].head()

In [ ]:
final_df.head()

In [ ]:
results = final_df[['post_code', 'bnf_name', 'amt%']]
results.head()

In [ ]:
results = results.head(100)
values = results.values.tolist()

In [ ]:
items_by_region = []

for item in values:
    items_by_region.append(tuple(item))
    
print(items_by_region)

In [ ]:
grader.score.dw__items_by_region(items_by_region)

## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [ ]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [ ]:
opioids_per_practice = ...

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [ ]:
relative_opioids_per_practice = ...

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [ ]:
standard_error_per_practice = ...
opioid_scores = ...

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
unique_practices = ...
anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [ ]:
import math
import gzip
import pandas as pd
from static_grader import grader


chem = pd.read_csv('./dw-data/chem.csv.gz', compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)
chem.head()
chem.columns

with gzip.open ( './dw-data/201701scripts_sample.csv.gz', 'rb' ) as f:
    scripts = pd.read_csv ( f )

with gzip.open ( './dw-data/practices.csv.gz', 'rb' ) as f:
    practices = pd.read_csv ( f )



practices.columns = ['code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code'] 
practices = practices[['code', 'name']].sort_values (by = ['name'], ascending = True) 
practices = practices [~practices.duplicated(['code'])] 
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine'] 


check = '|'.join(opioids) 
chem_df1 = chem 
chem_df1 [ 'test' ] = chem_df1 [ 'NAME' ].apply ( lambda x: any ( [ k in x.lower() for k in opioids ] ) ) 
key2 = chem_df1 [ "test" ] == True 
chem_df1 = chem_df1 [ key2 ]  
chem_sub = list (chem_df1['CHEM SUB']) 


scripts['opioid'] = scripts [ 'bnf_code' ].apply(lambda x: 1 if x in chem_sub else 0)
std_devn = scripts.opioid.std ()
overall_rate = scripts.opioid.mean()

scripts = scripts.merge (practices, left_on = 'practice', right_on = 'code')
scripts['cnt'] = 0


opioids_per_practice = scripts.groupby ( [ 'practice', 'name' ], as_index = False ).agg ( { 'opioid': 'mean', 'cnt': 'count' } )
opioids_per_practice.drop_duplicates()

opioids_per_practice['opioid'] = opioids_per_practice ['opioid'] - overall_rate

opioids_per_practice['std_err'] = std_devn / opioids_per_practice['cnt'] ** 0.5
opioids_per_practice['z_score'] = opioids_per_practice['opioid'] / opioids_per_practice['std_err']

result = opioids_per_practice[['name', 'z_score', 'cnt']]


result.sort_values(by = 'z_score', ascending = False, inplace = True)
anomalies = [(k[1], k[2], k[3]) for k in result.itertuples()][:100]

In [ ]:
grader.score.dw__script_anomalies(anomalies)

## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [ ]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz',compression='gzip', delimiter=',')
drugs_16 = scripts16[['bnf_name', 'items']]
drugs_16 = drugs_16.groupby('bnf_name').count().reset_index()
drugs_16.columns = ['bnf_name', 'count16']
drugs_16.head()

In [ ]:
drugs_17 = scripts[['bnf_name', 'items']]
drugs_17 = drugs_17.groupby('bnf_name').count().reset_index()
drugs_17.columns = ['bnf_name', 'count17']
drugs_17.head()

In [ ]:
drugs = drugs_16.merge(drugs_17, on='bnf_name', how='inner')
drugs = drugs[drugs['count16']>=50]

In [ ]:
drugs.is_copy = False
drugs['growth'] = ((drugs['count17']-drugs['count16'])/drugs['count16'])
drugs = drugs[['bnf_name', 'growth', 'count16']]
drugs.sort_values('growth', ascending=False, inplace=True)
drugs_final = pd.concat([drugs.iloc[:50], drugs.iloc[-50:]], axis=0)

In [ ]:
script_growth = drugs_final

In [ ]:
grader.score.dw__script_growth(script_growth)

## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [ ]:
p = ...
rates = ...
rare_codes = ...
scripts['rare'] = ...

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [ ]:
rare_cost_prop = ...

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [ ]:
relative_rare_cost_prop = ...

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [ ]:
standard_errors = ...

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
rare_scores = ...

In [ ]:
rates = scripts['bnf_code'].value_counts() / scripts['bnf_name'].count()

rates.head()

p = 1. /scripts['bnf_code'].nunique()

mask = rates < .1 * p

rare_codes = rates[mask].index

scripts['rare'] = scripts['bnf_code'].isin(rare_codes)

scripts.head()

rare_cost_prop = (scripts[scripts['rare']].groupby('practice')['act_cost'].sum()/ scripts.groupby('practice')['act_cost'].sum()).fillna(0)

rare_cost_prop.head()

cost_all = scripts[scripts['rare']]['act_cost'].sum() / scripts['act_cost'].sum()

relative_rare_cost_prop = rare_cost_prop - cost_all

standard_errors = relative_rare_cost_prop.std()  

z_score = relative_rare_cost_prop / standard_errors

In [ ]:
z_score = pd.DataFrame(z_score.sort_values(ascending = False))    

z_score.reset_index(inplace = True)

z_score.columns = ['practice', 'z_score']

fin = (practices.groupby(['code'])[['code', 'name']]).head() # 

result = z_score.merge(fin, how = 'left', left_on = 'practice',right_on = 'code').drop('code', axis = 1)

df = result.groupby('practice').first().sort_values('z_score', ascending = False).reset_index()[:100] 

In [ ]:
rare_scripts = list(zip(df.practice, df.name, df.z_score))

In [ ]:
grader.score.dw__rare_scripts(rare_scripts)

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*